## AgentExecutor (ReAct Agent — แบบเก่า/Legacy)

เอเจนต์จะวนลูป Reason → Act(tool) → Observe → อัตโนมัติ และเลือกใช้ search หรือ calculate_math_tool ได้เองหลายรอบ แต่แนวนี้ถูกจัดเป็น legacy และมีคำแนะนำให้ย้ายไป LangGraph หากงานซับซ้อนขึ้น

### Build Tool Function

In [ ]:
# from langchain_core.tools import tool

# @tool
# def calculate_math_tool(expression: str):
#     """Calculate mathematical expressions safely"""
#     try:
#         # Remove any potentially dangerous characters and evaluate safely
#         allowed_chars = set('0123456789+-*/(). ')
#         if all(c in allowed_chars for c in expression):
#             result = eval(expression)
#             return f"ผลลัพธ์: {expression} = {result}"
#         else:
#             return "ข้อผิดพลาด: นิพจน์ไม่ถูกต้อง ใช้ได้เฉพาะตัวเลขและเครื่องหมาย +, -, *, /, (, ) เท่านั้น"
#     except Exception as e:
#         return f"ข้อผิดพลาดในการคำนวณ: {str(e)}"
    
# calculate_math_tool.invoke("1+2")

'ผลลัพธ์: 1+2 = 3'

In [22]:
from langchain.agents import Tool

def calculate_math_tool(expression: str):
    """Calculate mathematical expressions safely"""
    try:
        allowed_chars = set('0123456789+-*/(). ')
        if not all(c in allowed_chars for c in expression):
            return "ข้อผิดพลาด: นิพจน์ไม่ถูกต้อง"
        return str(eval(expression))
    except Exception as e:
        return f"ข้อผิดพลาดในการคำนวณ: {str(e)}"

calculate_tool = Tool(
    name="calculator",
    func=calculate_math_tool,
    description="Use this to calculate mathematical expressions. Input like '2 + 3 * 4'."
)

calculate_tool.invoke("1+2")

'3'

In [ ]:
from langchain_community.utilities import GoogleSerperAPIWrapper

serper = GoogleSerperAPIWrapper()

search_tool = Tool(
    name="search",
    description="Useful for when you need more information from an online search",
    func=serper.run
)

### Create Agent

- create_tool_calling_agent -> Agent แบบ function/tool calling (เช่น Gemini, GPT-4)
- create_openai_functions_agent -> สำหรับ OpenAI function calling โดยเฉพาะ
- create_structured_chat_agent -> แบบ structured output (ใช้ JSON schema)
- create_react_agent -> Agent แบบ ReAct (reasoning + action)
- create_self_ask_with_search_agent -> Agent ที่ถามซ้อนและค้นข้อมูล
- create_xml_agent -> Agent ที่รับ/ส่งข้อมูลในรูปแบบ XML
- create_multi_function_agent -> Agent ที่เรียกหลาย tool พร้อมกัน (multi-action agent)

In [ ]:
from langchain_ollama import ChatOllama
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOllama(model="gemma3", temperature=0)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. You have access to these tools:\n{tools}\n\n"
            "When you need a tool, choose from: {tool_names}.\n"
            "You MUST follow the EXACT ReAct format:\n"
            "Thought: I need to think about what to do\n"
            "Action: tool_name\n"
            "Action Input: input_for_tool\n"
            "Observation: result_from_tool\n"
            "Thought: I can now answer the question\n"
            "Final Answer: final response\n\n"
            "Always use this exact format with proper line breaks.",
        ),
        ("human", "{input}\n\n{agent_scratchpad}"),
    ]
)

TOOLS = [search_tool, calculate_tool]

# {tools}/{tool_names} จะถูกเติมให้อัตโนมัติ
agent = create_react_agent(
    llm, TOOLS, prompt=prompt
)  
executor = AgentExecutor(agent=agent, tools=TOOLS, verbose=True, max_iterations=5)

In [18]:
print(executor.invoke({"input": "คำนวณ (6+2)*3 พร้อมคำอธิบาย"})["output"])



> Entering new AgentExecutor chain...
Thought: I need to calculate the expression (6+2)*3 and explain the steps.
Action: calculator
Action Input: (6+2)*324Thought: I can now answer the question.
Final Answer: (6+2)*3 = 8 * 3 = 24 คำนวณโดยเริ่มจากการบวก 6 และ 2 ซึ่งได้ผลลัพธ์เป็น 8 จากนั้นคูณ 8 ด้วย 3 ซึ่งได้ผลลัพธ์เป็น 24

> Finished chain.
(6+2)*3 = 8 * 3 = 24 คำนวณโดยเริ่มจากการบวก 6 และ 2 ซึ่งได้ผลลัพธ์เป็น 8 จากนั้นคูณ 8 ด้วย 3 ซึ่งได้ผลลัพธ์เป็น 24


In [19]:
print(executor.invoke({"input": "หาข้อมูลข่าว AMD"})["output"])



> Entering new AgentExecutor chain...
Thought: I need to search for recent news about AMD to provide a helpful response.
Action: search
Action Input: "AMD news"AMD stock earned a fresh buy rating after a Wall Street analyst spoke with industry contacts about adoption of the company's AI chips. AMD delivers leadership high-performance and adaptive computing solutions to advance data center AI, AI PCs, intelligent edge devices, gaming, & beyond. The most talked about and market moving research calls around Wall Street are now in one place. Here are today's research calls that ... See what's new with AMD. Read our blogs, catch up on the latest press releases, and discover our media resources. Get Advanced Micro Devices Inc (AMD:NASDAQ) real-time stock quotes, news, price and financial information from CNBC. Shares of computer processor maker AMD (NASDAQ:AMD) jumped 3.9% in the morning session after the company received a key analyst upgrade and ... AMD says it has boosted partner fundin